### Information Security Project

In [10]:
# ---------------- Hill Cipher ----------------

In [1]:
# ! .\isProjectEnv\Scripts\activate
# .\isProjectEnv\Scripts\activate
#imports
import numpy as np
from sympy import Matrix
import random
from collections import Counter

In [2]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
char_to_num = {char: i for i, char in enumerate(alphabet)}
num_to_char = {i: char for i, char in enumerate(alphabet)}

In [3]:

# def getKeyMatrix(key, n):
#     """
#     Generates the key matrix for the Hill cipher.
#     """
#     k = 0
#     keyMatrix = [[0] * n for i in range(n)]
#     for i in range(n):
#         for j in range(n):
#             keyMatrix[i][j] = ord(key[k]) % 65
#             k += 1
#     return np.array(keyMatrix)

# def padding(plaintext, n):
#     """
#     Pads the plaintext with 'X' to make its length a multiple of n.
#     Returns padded plaintext and number of padding chars added.
#     """
#     plaintext = plaintext.replace(" ", "").upper()
#     pad_len = (n - len(plaintext) % n) % n
#     plaintext += 'X' * pad_len
#     return plaintext, pad_len

# def encrypt(plaintext_blocks, key_matrix):
#     """
#     Encrypts plaintext blocks using the Hill cipher.
#     """
#     ciphertext = ""
#     for block in plaintext_blocks:
#         encrypted_block = np.dot(key_matrix, block) % 26
#         for num in encrypted_block:
#             ciphertext += num_to_char[num[0]]
#     return ciphertext

# def modMatInv(matrix, modulus):
#     """
#     Computes modular inverse of a matrix under modulus.
#     Uses sympy for adjugate + determinant.
#     """
#     M = Matrix(matrix)
#     det = int(M.det()) % modulus
#     if np.gcd(det, modulus) != 1:
#         raise ValueError("Key matrix not invertible modulo {}".format(modulus))
#     det_inv = pow(det, -1, modulus)
#     adjugate = M.adjugate()
#     return np.array((det_inv * adjugate) % modulus).astype(int)

# def decrypt(cipher_blocks, inverse_key_matrix, pad_len=0):
#     """
#     Decrypts ciphertext blocks using the Hill cipher.
#     Removes padding automatically.
#     """
#     decrypted_text = ""
#     for block in cipher_blocks:
#         decrypted_block = np.dot(inverse_key_matrix, block) % 26
#         for num in decrypted_block:
#             decrypted_text += num_to_char[num[0]]
#     if pad_len > 0:
#         decrypted_text = decrypted_text[:-pad_len]  # strip padding
#     return decrypted_text


# # ------------------------------
# # Example run
# # ------------------------------

# # Key matrix
# key_matrix = getKeyMatrix("GYBNQKURP", 3)
# print("Key Matrix:\n", key_matrix)

# # Plaintext
# # plaintext = "Quantifying Confusion and Diffusion"
# plaintext = "AdistributedsystemisacollectionofindependententitiesthatcooperatetosolveaproblemthatcannotbeindividuallysolvedDistributedsystemshavebeeninexistencesincethestartoftheuniverseFromaschooloffishtoaflockofbirdsandentireecosystemsofmicroorganismsthereiscommunicationamongmobileintelligentagentsinnatureWiththewidespreadproliferationoftheinternetandtheemergingglobalvillagethenotionofdistributedcomputingsystemsasausefulandwidelydeployedtoolisbecomingareality"
# plaintext, pad_len = padding(plaintext, 3)
# plaintext_block = np.array([[char_to_num[char]] for char in plaintext])
# plaintext_blocks = [plaintext_block[i:i+3] for i in range(0, len(plaintext_block), 3)]
# print("\nPadded Plaintext:", plaintext)

# # Encryption
# ciphertext = encrypt(plaintext_blocks, key_matrix)
# print("Ciphertext:", ciphertext)

# # Ciphertext to numeric blocks
# encrypted_block = np.array([[char_to_num[char]] for char in ciphertext])
# cipher_blocks = [encrypted_block[i:i+3] for i in range(0, len(encrypted_block), 3)]

# # Inverse key matrix
# inverse_key_matrix = modMatInv(key_matrix, 26)
# print("\nInverse Key Matrix:\n", inverse_key_matrix)

# # Decryption (auto removes padding)
# decrypted_text = decrypt(cipher_blocks, inverse_key_matrix, pad_len)
# print("Decrypted Text:", decrypted_text)


In [4]:
class HillCipher:
    def __init__(self, key, n):
        self.alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.char_to_num = {char: i for i, char in enumerate(self.alphabet)}
        self.num_to_char = {i: char for i, char in enumerate(self.alphabet)}
        self.n = n
        self.key = key
        self.key_matrix = self.getKeyMatrix(key, n)
        self.inverse_key_matrix = self.modMatInv(self.key_matrix, 26)

    def getKeyMatrix(self, key, n):
        """
        Generates the key matrix for the Hill cipher.
        """
        k = 0
        keyMatrix = [[0] * n for _ in range(n)]
        for i in range(n):
            for j in range(n):
                keyMatrix[i][j] = ord(key[k]) % 65
                k += 1
        return np.array(keyMatrix)

    def padding(self, plaintext):
        """
        Pads the plaintext with 'X' to make its length a multiple of n.
        Returns padded plaintext and number of padding chars added.
        """
        plaintext = plaintext.replace(" ", "").upper()
        pad_len = (self.n - len(plaintext) % self.n) % self.n
        plaintext += 'X' * pad_len
        return plaintext, pad_len

    def encrypt(self, plaintext):
        """
        Encrypts plaintext blocks using the Hill cipher.
        """
        plaintext, pad_len = self.padding(plaintext)
        blocks = np.array([[self.char_to_num[char]] for char in plaintext])
        plaintext_blocks = [blocks[i:i+self.n] for i in range(0, len(blocks), self.n)]
        ciphertext = ""
        for block in plaintext_blocks:
            encrypted_block = np.dot(self.key_matrix, block) % 26
            for num in encrypted_block:
                ciphertext += self.num_to_char[num[0]]
        return ciphertext, pad_len

    def modMatInv(self, matrix, modulus):
        """
        Computes modular inverse of a matrix under modulus.
        Uses sympy for adjugate + determinant.
        """
        M = Matrix(matrix)
        det = int(M.det()) % modulus
        if np.gcd(det, modulus) != 1:
            raise ValueError("Key matrix not invertible modulo {}".format(modulus))
        det_inv = pow(det, -1, modulus)
        adjugate = M.adjugate()
        return np.array((det_inv * adjugate) % modulus).astype(int)

    def decrypt(self, ciphertext, pad_len=0):
        """
        Decrypts ciphertext blocks using the Hill cipher.
        Removes padding automatically.
        """
        blocks = np.array([[self.char_to_num[char]] for char in ciphertext])
        cipher_blocks = [blocks[i:i+self.n] for i in range(0, len(blocks), self.n)]
        decrypted_text = ""
        for block in cipher_blocks:
            decrypted_block = np.dot(self.inverse_key_matrix, block) % 26
            for num in decrypted_block:
                decrypted_text += self.num_to_char[num[0]]
        if pad_len > 0:
            decrypted_text = decrypted_text[:-pad_len]
        return decrypted_text
    def getCiphertext(self, plaintext):
        """
        Encrypts the given plaintext and returns the ciphertext and padding length.
        """
        ciphertext, pad_len = self.encrypt(plaintext)
        return ciphertext, pad_len

In [5]:
hilCipherObject = HillCipher("GYBNQKURP", 3)
# 10 invertible key strings of length 9 can be used for 3x3 matrix


In [6]:
plaintext_1 = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
plaintext_2 = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB"
plaintext_3 = "AdistributedsystemisacollectionofindependententitiesthatcooperatetosolveaproblemthatcannotbeindividuallysolvedDistributedsystemshavebeeninexistencesincethestartoftheuniverseFromaschooloffishtoaflockofbirdsandentireecosystemsofmicroorganismsthereiscommunicationamongmobileintelligentagentsinnatureWiththewidespreadproliferationoftheinternetandtheemergingglobalvillagethenotionofdistributedcomputingsystemsasausefulandwidelydeployedtoolisbecomingareality"
# random plaintext of length 453 using alpahbet and random.choices
plaintext_4 =  ''.join(random.choices(alphabet, k=453))
plaintext_array = [plaintext_1, plaintext_2, plaintext_3, plaintext_4]

In [ ]:
# modified_ciphertext_1, _ = hilCipherObject.getCiphertext(plaintext_3)
# print("Ciphertext 1:", modified_ciphertext_1)
# mcl1 = list()
# plaintext_3[0] = 'C'
# plaintext_3[1] = 'f'
# plaintext_3[2] = 'k'
# plaintext_3[3] = 'u'
# plaintext_3[4] = 'v'
# modified_ciphertext_2, _ = hilCipherObject.getCiphertext(plaintext_3)
# print("Ciphertext 2:", modified_ciphertext_2)

Ciphertext 1: CYPJGCOIJFDZASYOPEMCYVWBITGIVYDBBZEKYUFYXHSTTWHBYZEZKIQMPTKPQMFVHFKMQCZNNUQPWUTSXJVJZABRLDGBACLPJPDBGTKFOFWCVAUFKXJVXIGDGVYWGPRUBOMBCNSXLTMGTWGPAHQBYPXUDAXKBBLNBRAJNYCNFNPBKJITESWYCNQRBNGJMSPXBIQMCWXMQRQARAJXZLLZRSGWUIKWGPYMCOZISWTEUJICDYICAJNYBIOQGQCQMFKWUBGAHFQXAUOBRJVYVNFPETERIOSGGGDVJAEIKZMLKRKQAJNPCXDWNQZSFHYSWPNNHWUBUKJZXIVYVTNBLHPDEGAJNEYGUHQUNKNNFZWILASSGGAJNRLDHQVZGUFKXJVXIGDCDSKYPHLJYIGWGPKCAYSKBKJUUPQVRUKLVLXFMGOIUUPVSWPGEVWGQCIDWALMWVLDP


TypeError: 'str' object does not support item assignment

In [7]:
#Over fixed plaintext

# experimenting avalanche effect make a small change to the plaintext (an "incremental change") and re-encrypt it. The goal is to measure the 
#avalanche effect, which is the fraction of ciphertext symbols that change due to this small plaintext modification.  
# Then repeat this experiment with multiple positions and plaintexts and then calculate the average result

def measure_avalanche_effect(hill_cipher, original_plaintext, num_trials=100):
        total_changed_chars = 0
        original_ciphertext, _ = hill_cipher.getCiphertext(original_plaintext)
        
        for _ in range(num_trials):
            # Randomly change one character in the plaintext
            pos = random.randint(0, len(original_plaintext) - 1)
            modified_plaintext = list(original_plaintext)
            modified_plaintext[pos] = random.choice(hill_cipher.alphabet.replace(modified_plaintext[pos], ''))
            modified_plaintext = ''.join(modified_plaintext)        

            # Encrypt the modified plaintext
            modified_ciphertext, _ = hill_cipher.getCiphertext(modified_plaintext)
            
            # Count the number of differing characters in the ciphertexts
            changed_chars = sum(1 for a, b in zip(original_ciphertext, modified_ciphertext) if a != b)
            total_changed_chars += changed_chars
        
        average_changed_chars = total_changed_chars / num_trials
        avalanche_effect = average_changed_chars / len(original_ciphertext)
        return avalanche_effect

# plaintext = "Quantifying Confusion and Diffusion"
plaintrext_array = [plaintext_1, plaintext_2, plaintext_3, plaintext_4]
for plaintext_i in plaintext_array:
    ciphertext_i, pad_len = hilCipherObject.getCiphertext(plaintext_i)
    print("\nPlaintext:", plaintext_i)
    print("Ciphertext from class:", ciphertext_i)        
    
    avalanche_effect = measure_avalanche_effect(hilCipherObject, plaintext_i)
    print(f"Avalanche Effect: {avalanche_effect:.2%}")


Plaintext: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Ciphertext from class: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAXWH
Avalanche Effect: 0.64%

Plaintext: AAAAAAAAAAAAAAAAAAAAAA

In [8]:
#Over fixed plaintext

#To quantify confusion, you'll encrypt a fixed plaintext using a specific key. Then, make an incremental change to the key and re-encrypt the same plaintext. 
# You'll then measure the resulting change in the ciphertext and report the average changes. This process helps measure the cipher's 
#confusion property by observing how sensitive the ciphertext is to small changes in the key.
def measure_confusion(hill_cipher, original_plaintext, num_trials=100):
    total_changed_chars = 0
    original_ciphertext, _ = hill_cipher.getCiphertext(original_plaintext)
    
    for _ in range(num_trials):
        # Randomly change one character in the key
        pos = random.randint(0, len(hill_cipher.key) - 1)
        modified_key = list(hill_cipher.key)
        #removing all occurrences of the character at the specified position in alphabet so that we don't get the same character again
        modified_key[pos] = random.choice(hill_cipher.alphabet.replace(modified_key[pos], '')) 
        modified_key = ''.join(modified_key)[:hill_cipher.n * hill_cipher.n]  # Ensure key length matches
        
        # use getKeyMatrix to get the key matrix and check if it is invertible mod 26
        try:
            modified_key_matrix = hill_cipher.getKeyMatrix(modified_key, hill_cipher.n)
            _ = hill_cipher.modMatInv(modified_key_matrix, 26)  # Check invertibility
        except ValueError:
            continue  # Skip if not invertible
        

        # Create a new HillCipher object with the modified key
        modified_hill_cipher = HillCipher(modified_key, hill_cipher.n)
        
        # Encrypt the same plaintext with the modified key
        modified_ciphertext, _ = modified_hill_cipher.getCiphertext(original_plaintext)
        
        # Count the number of differing characters in the ciphertexts
        changed_chars = sum(1 for a, b in zip(original_ciphertext, modified_ciphertext) if a != b)
        total_changed_chars += changed_chars
    
    average_changed_chars = total_changed_chars / num_trials
    confusion_effect = average_changed_chars / len(original_ciphertext)
    return confusion_effect

for plaintext_j in plaintext_array:
    confusion_effect = measure_confusion(hilCipherObject, plaintext_j)
    print(f"Confusion Effect: {confusion_effect:.2%}")

Confusion Effect: 0.07%
Confusion Effect: 9.97%
Confusion Effect: 22.10%
Confusion Effect: 19.81%


In [9]:
#calculating shannon entropy of the ciphertext
from collections import Counter 
def shannon_entropy(text):
    """
    Calculate the Shannon entropy of a given text.
    """
    if not text:
        return 0
    frequency = Counter(text)
    probabilities = [freq / len(text) for freq in frequency.values()]
    entropy = -sum(p * np.log2(p) for p in probabilities if p > 0)
    return entropy
# ciphertext_entropy = shannon_entropy(ciphertext)
# print(f"Shannon Entropy of Ciphertext: {ciphertext_entropy:.4f} bits per character")

#measure shannon entropy for each plaintext
for plaintext_m in plaintext_array:
    ciphertext_m, _ = hilCipherObject.getCiphertext(plaintext_m)
    ciphertext_entropy = shannon_entropy(ciphertext_m)
    print(f"Shannon Entropy of Ciphertext for given plaintext: {ciphertext_entropy:.4f} bits per character")


Shannon Entropy of Ciphertext for given plaintext: 0.0680 bits per character
Shannon Entropy of Ciphertext for given plaintext: 1.2365 bits per character
Shannon Entropy of Ciphertext for given plaintext: 4.6532 bits per character
Shannon Entropy of Ciphertext for given plaintext: 4.6653 bits per character


In [10]:
#calculating mutual information between plaintext and ciphertext
def mutual_information(plaintext, ciphertext):  
    """
    Calculate the mutual information between plaintext and ciphertext.
    """
    if not plaintext or not ciphertext or len(plaintext) != len(ciphertext):
        return 0

    joint_freq = Counter(zip(plaintext, ciphertext))
    p_x = Counter(plaintext)
    p_y = Counter(ciphertext)
    
    total_chars = len(plaintext)
    
    mi = 0
    for (x, y), joint_count in joint_freq.items():
        p_xy = joint_count / total_chars
        p_x_val = p_x[x] / total_chars
        p_y_val = p_y[y] / total_chars
        mi += p_xy * np.log2(p_xy / (p_x_val * p_y_val))
    
    return mi
# old_plaintext = "Quantifying Confusion and Diffusion"
# mi = mutual_information(plaintext, ciphertext)
# print(f"Mutual Information between Plaintext and Ciphertext: {mi:.4f} bits")

#meaure mutual information for each plaintext
for plaintext_k in plaintext_array:
    ciphertext_k, _ = hilCipherObject.getCiphertext(plaintext_k)
    mi = mutual_information(plaintext_k, ciphertext_k)
    print(f"Mutual Information between Plaintext and Ciphertext: {mi:.4f} bits")

Mutual Information between Plaintext and Ciphertext: 0.0000 bits
Mutual Information between Plaintext and Ciphertext: 0.4716 bits
Mutual Information between Plaintext and Ciphertext: 0.0000 bits
Mutual Information between Plaintext and Ciphertext: 1.0696 bits


In [11]:

def mutual_information_blocks(plaintext, ciphertext, block_size=1):
    """
    Calculate mutual information between plaintext and ciphertext
    at the block level.

    plaintext, ciphertext: strings of equal length
    block_size: number of symbols per block (e.g. 2 or 3 for Hill cipher)

    Returns: MI in bits
    """
    # Ensure valid inputs
    if not plaintext or not ciphertext or len(plaintext) != len(ciphertext):
        return 0
    
    # Trim to a multiple of block_size
    L = len(plaintext) - (len(plaintext) % block_size)
    if L == 0:
        return 0
    
    plaintext = plaintext[:L]
    ciphertext = ciphertext[:L]

    # Break into blocks
    p_blocks = [plaintext[i:i+block_size] for i in range(0, L, block_size)]
    c_blocks = [ciphertext[i:i+block_size] for i in range(0, L, block_size)]

    # Joint and marginal counts
    joint_freq = Counter(zip(p_blocks, c_blocks))
    p_freq = Counter(p_blocks)
    c_freq = Counter(c_blocks)
    
    total_blocks = len(p_blocks)
    mi = 0.0

    for (p, c), joint_count in joint_freq.items():
        p_xy = joint_count / total_blocks
        p_x = p_freq[p] / total_blocks
        p_y = c_freq[c] / total_blocks
        mi += p_xy * np.log2(p_xy / (p_x * p_y))
    
    return mi


# measure mutual information at block level for each plaintext
for plaintext_l in plaintext_array:
    ciphertext_l, _ = hilCipherObject.getCiphertext(plaintext_l)        
    # print("MI (block_size=1):", mutual_information_blocks(plaintext_l, ciphertext_l, block_size=1))
    print("MI (block_size=3):", mutual_information_blocks(plaintext_l, ciphertext_l, block_size=3))
    print()

MI (block_size=3): 0

MI (block_size=3): 1.0426679938782875

MI (block_size=3): 0

MI (block_size=3): 7.225159706212477



In [12]:
from cdci_score_module import cdci_score
# Use cdci module to calculate score over plaintext_array


TypeError: cdci_score() missing 3 required positional arguments: 'key', 'keyspace', and 'alphabet'

In [ ]:
import secretpy as spy
import pycipher as pc